In [ ]:
%pip install azure-eventhub

#### Load ML Model

In [ ]:
%run TrainingModel

In [3]:
def clean_key(key):
    # Remove spaces
    key = key.replace(' ', '')
    # Remove square brackets and anything inside them
    key = re.sub(r'\[.*?\]', '', key)
    return key

StatementMeta(, , , Waiting, , Waiting)

#### Simulate The Data

In [ ]:
import time
import json
import numpy as np
import re
from datetime import datetime
from azure.eventhub import EventHubProducerClient, EventData
 
# Define Event Hub connection
connection_str = ''
producer = EventHubProducerClient.from_connection_string(connection_str)
 
# Predefine machine types and safer initial values
num_machines = 100
types = ['L', 'M', 'H']
machines = []
 
# Initialize machines with predefined, safer initial values
for machine_id in range(1, num_machines + 1):
    machine_type = np.random.choice(types)
    # Set initial values that are within a safe range
    initial_air_temp = 295 + np.random.uniform(0, 5)  # Start below critical temperature
    initial_rotational_speed = 1450 + np.random.uniform(-20, 20)  # Start below high RPM
    machines.append({
        'Machine ID': machine_id,
        'Type': machine_type,
        'Air temperature [K]': initial_air_temp,
        'Process temperature [K]': initial_air_temp + 20,
        'Rotational speed [rpm]': initial_rotational_speed,
        'Torque [Nm]': 40 + np.random.uniform(0, 5),  # Start with moderate torque
        'Tool wear [min]': np.random.randint(0, 50)  # Start with low tool wear
    })

# Simulate data and send to Event Hub
while True:
    event_data_batch = producer.create_batch()
 
    for machine in machines:
        # Simulate gradual changes over time
        machine['Air temperature [K]'] += np.random.uniform(0.1, 0.5)
        machine['Process temperature [K]'] = machine['Air temperature [K]'] + np.random.uniform(15, 25)
        machine['Rotational speed [rpm]'] += np.random.uniform(1, 5)
        machine['Torque [Nm]'] += np.random.uniform(0.1, 0.3)
        machine['Tool wear [min]'] += 0.2 if machine['Type'] == 'L' else 0.4 if machine['Type'] == 'M' else 0.6
 
        # Determine failure type based on conditions
        if (machine['Air temperature [K]'] <= 310 and 
            machine['Process temperature [K]'] <= 320 and 
            machine['Rotational speed [rpm]'] <= 1600 and 
            machine['Tool wear [min]'] < 150):
            failure_type = "No Failure"
        elif (machine['Air temperature [K]'] > 320 and 
              machine['Process temperature [K]'] > 330 and 
              machine['Torque [Nm]'] < 35):
            failure_type = "Heat Dissipation Failure"
        elif (machine['Rotational speed [rpm]'] > 1700 and 
              machine['Torque [Nm]'] > 50 and 
              325 <= machine['Process temperature [K]'] <= 335):
            failure_type = "Power Failure"
        elif (machine['Rotational speed [rpm]'] > 1700 and 
              machine['Tool wear [min]'] > 200):
            failure_type = "Overstrain Failure"
        elif (machine['Tool wear [min]'] > 250 and 
              1500 <= machine['Rotational speed [rpm]'] <= 1700):
            failure_type = "Tool Wear Failure"
        else:
            failure_type = "Random Failures" if np.random.rand() < 0.1 else "No Failure"
 
        # Encode failure type for model use
        target = ["No Failure", "Heat Dissipation Failure", "Power Failure", 
                  "Overstrain Failure", "Tool Wear Failure", "Random Failures"].index(failure_type)
 
        # Prepare data payload
        machine_data = {
            'Machine ID': machine['Machine ID'],
            'Type': machine['Type'],
            'Air temperature [K]': round(machine['Air temperature [K]'], 2),
            'Process temperature [K]': round(machine['Process temperature [K]'], 2),
            'Rotational speed [rpm]': round(machine['Rotational speed [rpm]'], 2),
            'Torque [Nm]': round(machine['Torque [Nm]'], 2),
            'Tool wear [min]': round(machine['Tool wear [min]'], 2),
            'Failure Type': failure_type,
            'Target': target,
            'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'uid': str(machine['Machine ID'])+'||'+datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        # final_df=mloutput([machine_data])
        # dict_data = final_df.iloc[0].to_dict() 
        dict_data=machine_data

        # Clean and convert data to JSON
        cleaned_data = {clean_key(key): value for key, value in dict_data.items()}
        json_data = json.dumps(cleaned_data)
        event_data_batch.add(EventData(json_data))
 
        # Print the generated data for verification
        print(json_data)
 
    # Send the batch of data to Event Hub
    producer.send_batch(event_data_batch)
 
    # Wait 5 seconds before generating the next set of data
    time.sleep(5)

